<a id="inicio-notebook"></a>
# Proyecto End to End de Machine Learning 
### Viviendas en venta en Madrid


## 0. Librerías
 

In [1]:
# importación agrupada de librerías necesarias en este notebook
import pandas as pd
from pandas import StringDtype
import numpy as np
import json
import re

import sys
import os
from datetime import date

from scipy import stats
from scipy.stats import chi2_contingency
from PIL import Image
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, LabelBinarizer, MultiLabelBinarizer, OneHotEncoder 
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

#warnings.filterwarnings('ignore')

# Añado el directorio padre (del que está este notebook) a sys.path
sys.path.append(os.path.abspath('../'))
from scripts.utils_agv import ini_inspec, crear_tabla_resumen, categoricas, numericas

<a id="comprension-variables"></a>
## 4. Compresión de variables

In [2]:
# carga de los datos guardados en el anterior paso
df = pd.read_csv('../data/processed/ide_viv_limpieza0_2025-03-11.csv', index_col='propertyCode')

In [3]:
df.index.name = 'ID'
# Mover la columna 'price' a la primera posición
columnas = ['price'] + [col for col in df.columns if col != 'price']
df = df.reindex(columns=columnas)
print (df.shape)
df.head(1)

(1747, 24)


,price,numPhotos,floor,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasVideo,status,hasLift,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,topPlus
ID,,,,,,,,,,,,,,,,,,,,,
107526421,1095000.0,43,3,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,good,True,7500.0,{'typology': 'flat'},"{'subtitle': 'Sol, Madrid', 'title': 'Piso en ...",True,True,False,True


In [4]:
ini_inspec(df)

=== TAMAÑO Y ESTRUCTURA DE LOS DATOS ===
Número total de registros (filas): 1747
Número de columnas: 24
Uso de memoria: 281.50 KB


=== TIPOS DE DATOS Y NOMBRES DE COLUMNAS ===
price             float64
numPhotos           int64
floor              object
propertyType       object
size              float64
exterior           object
rooms               int64
bathrooms           int64
address            object
district           object
neighborhood       object
latitude          float64
longitude         float64
description        object
hasVideo             bool
status             object
hasLift            object
priceByArea       float64
detailedType       object
suggestedTexts     object
hasPlan              bool
has3DTour            bool
has360               bool
topPlus              bool
dtype: object


Información detallada del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1747 entries, 107526421 to 107518858
Data columns (total 24 columns):
 #   Column          Non-Null C

Después de esta primera inspección, confirmado que no hay duplicados, vamos a abordar los problemas que observo: valores faltantes, contenidos como diccionario.

### Tratamiento (desdoblado) de las columnas cuyos valores son diccionarios

In [5]:
df['detailedType'].unique()

array(["{'typology': 'flat'}",
       "{'typology': 'flat', 'subTypology': 'studio'}",
       "{'typology': 'flat', 'subTypology': 'penthouse'}",
       "{'typology': 'flat', 'subTypology': 'duplex'}"], dtype=object)

In [6]:
df['suggestedTexts'].unique()

array(["{'subtitle': 'Sol, Madrid', 'title': 'Piso en calle de las Fuentes'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Divino Pastor'}",
       "{'subtitle': 'Lavapiés-Embajadores, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso en Carrera de San Jerónimo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Estudio en calle de San Andrés'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Ático en calle de Fuencarral'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle de Fuencarral'}",
       "{'subtitle': 'Palacio, Madrid', 'title': 'Dúplex en calle de Santiago'}",
       "{'subtitle': 'Chueca-Justicia, Madrid', 'title': 'Piso en calle de Apodaca'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Espíritu Santo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 't

Voy a separar en columnas aquellas cuyos datos son dicionarios:


In [7]:
#FUNCIÓN PARA EXPANDIR CELDAS CON CONTENIDO DICCIONARIOS
def expand_dict_columns(df):
    """
    Expande las columnas del dataframe de Idealista que contienen diccionarios.
    
    Parámetros:
    df (pandas.DataFrame): DataFrame con datos de Idealista
    
    Retorna:
    pandas.DataFrame: DataFrame con las columnas expandidas
    """
    # Hacer una copia del dataframe original para no modificarlo
    df_processed = df.copy()
    
    def parse_dict_safely(value):
        """Convierte strings a diccionarios de forma segura sin usar ast"""
        if pd.isna(value):
            return {}
        if isinstance(value, dict):
            return value
        if isinstance(value, str) and value.strip():
            try:
                # Intentar convertir usando json.loads
                return json.loads(value)
            except json.JSONDecodeError:
                try:
                    # Si falla, corregimos comillas simples a dobles
                    value = value.replace("'", "\"")
                    return json.loads(value)
                except json.JSONDecodeError:
                    # Si aún falla, retornar vacío
                    return {}
        return {}
    
    def process_column(column_name, field_mappings):
        """
        Procesa una columna de diccionario y extrae campos específicos.
        
        Parámetros:
        column_name (str): Nombre de la columna a procesar
        field_mappings (dict): Diccionario donde la clave es el nombre del campo 
                               a extraer y el valor es un valor por defecto
        """
        if column_name not in df_processed.columns:
            return
            
        # Convertir strings a diccionarios
        df_processed[column_name] = df_processed[column_name].apply(parse_dict_safely)
        
        # Extraer cada campo del diccionario
        for field, default_value in field_mappings.items():
            new_column_name = f"{column_name}_{field}"
            df_processed[new_column_name] = df_processed[column_name].apply(
                lambda x: x.get(field, default_value) if isinstance(x, dict) else default_value
            )
    
    # Definir los campos a extraer para cada columna
    column_fields = {
        'suggestedTexts': {'subtitle': None, 'title': None},
        'detailedType': {'typology': None, 'subTypology': None},
        'parkingSpace': {
            'hasParkingSpace': False, 
            'isParkingSpaceIncludedInPrice': None,
            'parkingSpacePrice': None
        }
    }
    
    # Procesar cada columna
    for column, fields in column_fields.items():
        process_column(column, fields)
    
    # Eliminar las columnas originales
    columns_to_drop = [col for col in column_fields.keys() if col in df_processed.columns]
    df_processed = df_processed.drop(columns=columns_to_drop)
    
    return df_processed

In [8]:
# CELDA SALTADA intento de tratamiento de todas las columnas con diccionarios, incluso Parking
pass
def expand_dict_columns2(df):
    """
    Expande las columnas del dataframe de Idealista que contienen diccionarios.
    
    Parámetros:
    df (pandas.DataFrame): DataFrame con datos de Idealista
    
    Retorna:
    pandas.DataFrame: DataFrame con las columnas expandidas
    """
    # Hacer una copia del dataframe original para no modificarlo
    df_processed = df.copy()
    
    def parse_dict_safely(value):
        """Convierte strings a diccionarios de forma segura sin usar ast"""
        if pd.isna(value):
            return {}
        if isinstance(value, dict):
            return value
        if isinstance(value, str) and value.strip():
            try:
                # Intentar convertir usando json.loads
                return json.loads(value)
            except json.JSONDecodeError:
                try:
                    # Si falla, corregimos comillas simples a dobles
                    value = value.replace("'", "\"")
                    return json.loads(value)
                except json.JSONDecodeError:
                    # Si aún falla, retornar vacío
                    return {}
        return {}
    
    # Procesar columna suggestedTexts
    if 'suggestedTexts' in df_processed.columns:
        df_processed['suggestedTexts'] = df_processed['suggestedTexts'].apply(parse_dict_safely)
        df_processed['suggestedTexts_subtitle'] = df_processed['suggestedTexts'].apply(
            lambda x: x.get('subtitle') if isinstance(x, dict) else None
        )
        df_processed['suggestedTexts_title'] = df_processed['suggestedTexts'].apply(
            lambda x: x.get('title') if isinstance(x, dict) else None
        )
    
    # Procesar columna detailedType
    if 'detailedType' in df_processed.columns:
        df_processed['detailedType'] = df_processed['detailedType'].apply(parse_dict_safely)
        df_processed['detailedType_typology'] = df_processed['detailedType'].apply(
            lambda x: x.get('typology') if isinstance(x, dict) else None
        )
        df_processed['detailedType_subTypology'] = df_processed['detailedType'].apply(
            lambda x: x.get('subTypology') if isinstance(x, dict) else None
        )
    
    # Procesar columna parkingSpace - Esto lo dejamos explícito para manejar mejor los casos especiales
    if 'parkingSpace' in df_processed.columns:
        # Convertir strings a diccionarios y manejar valores NaN
        df_processed['parkingSpace'] = df_processed['parkingSpace'].apply(parse_dict_safely)
        
        # Extraer hasParkingSpace - valor por defecto es False
        df_processed['parkingSpace_hasParkingSpace'] = df_processed['parkingSpace'].apply(
            lambda x: x.get('hasParkingSpace', False) if isinstance(x, dict) else False
        )
        
        # Extraer isParkingSpaceIncludedInPrice - Sin valor por defecto para preservar NaN cuando no existe
        df_processed['parkingSpace_isParkingSpaceIncludedInPrice'] = df_processed['parkingSpace'].apply(
            lambda x: x.get('isParkingSpaceIncludedInPrice') if isinstance(x, dict) else pd.NA
        )
        
        # Extraer parkingSpacePrice - Convertimos a float explícitamente si existe
        df_processed['parkingSpace_parkingSpacePrice'] = df_processed['parkingSpace'].apply(
            lambda x: float(x.get('parkingSpacePrice')) if isinstance(x, dict) and 'parkingSpacePrice' in x and x['parkingSpacePrice'] is not None else pd.NA
        )
    
    # Eliminar las columnas originales que contenían diccionarios
    columns_to_drop = []
    for col in ['suggestedTexts', 'detailedType', 'parkingSpace']:
        if col in df_processed.columns:
            columns_to_drop.append(col)
    
    df_processed = df_processed.drop(columns=columns_to_drop)
    return df_processed

In [9]:
# df_ampliado_dict = expand_dict_columns(df, dict_columns=['otraColumnaDict', 'segundaColumnaDict'])
df_ampliado_dict = expand_dict_columns(df)
df_ampliado_dict

,price,numPhotos,floor,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasLift,priceByArea,hasPlan,has3DTour,has360,topPlus,suggestedTexts_subtitle,suggestedTexts_title,detailedType_typology,detailedType_subTypology
ID,,,,,,,,,,,,,,,,,,,,,
107526421,1095000.0,43,3,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,7500.0,True,True,False,True,"Sol, Madrid",Piso en calle de las Fuentes,flat,None
107246640,1195000.0,27,2,flat,134.0,True,3,3,calle del Divino Pastor,Centro,...,True,8918.0,True,True,True,True,"Malasaña-Universidad, Madrid",Piso en calle del Divino Pastor,flat,None
106773918,790000.0,33,3,flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,Centro,...,True,5338.0,True,True,False,True,"Lavapiés-Embajadores, Madrid",Piso,flat,None
102711209,2350000.0,44,5,flat,218.0,True,4,4,Carrera de San Jerónimo,Centro,...,True,10780.0,True,True,False,True,"Sol, Madrid",Piso en Carrera de San Jerónimo,flat,None
106822977,235000.0,34,bj,studio,38.0,False,0,1,calle de San Andrés,Centro,...,True,6184.0,True,True,False,True,"Malasaña-Universidad, Madrid",Estudio en calle de San Andrés,flat,studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107419060,4200000.0,0,3,flat,288.0,True,3,3,calle del Almirante,Centro,...,True,14583.0,False,False,False,False,"Chueca-Justicia, Madrid",Piso en calle del Almirante,flat,None
106976644,2810000.0,0,1,flat,238.0,True,3,4,Barrio Sol,Centro,...,True,11807.0,False,False,False,False,"Sol, Madrid",Piso,flat,None
107230033,2170000.0,0,4,penthouse,125.0,True,2,3,Barrio Chueca-Justicia,Centro,...,True,17360.0,False,False,False,False,"Chueca-Justicia, Madrid",Ático,flat,penthouse


In [10]:
# conversión Parking Space en multiples columnas bien: finalmente esto no lo he realizado, aunque hice multiples intentos, y nunca lograba que me funcionara bien
# TODO mejorar esto
# df_ampliado_dict['parkingSpace_hasParkingSpace'].unique()
# df_ampliado_dict['parkingSpace_isParkingSpaceIncludedInPrice'].unique()
# df_ampliado_dict['parkingSpace_parkingSpacePrice'].unique()

In [11]:
df2=df_ampliado_dict.copy()

In [12]:
df2['detailedType_typology'].unique()

array(['flat'], dtype=object)

In [13]:
df2['detailedType_subTypology'].unique()

array([None, 'studio', 'penthouse', 'duplex'], dtype=object)

In [14]:
df2.head()

,price,numPhotos,floor,propertyType,size,exterior,rooms,bathrooms,address,district,...,hasLift,priceByArea,hasPlan,has3DTour,has360,topPlus,suggestedTexts_subtitle,suggestedTexts_title,detailedType_typology,detailedType_subTypology
ID,,,,,,,,,,,,,,,,,,,,,
107526421,1095000.0,43,3,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,True,7500.0,True,True,False,True,"Sol, Madrid",Piso en calle de las Fuentes,flat,None
107246640,1195000.0,27,2,flat,134.0,True,3,3,calle del Divino Pastor,Centro,...,True,8918.0,True,True,True,True,"Malasaña-Universidad, Madrid",Piso en calle del Divino Pastor,flat,None
106773918,790000.0,33,3,flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,Centro,...,True,5338.0,True,True,False,True,"Lavapiés-Embajadores, Madrid",Piso,flat,None
102711209,2350000.0,44,5,flat,218.0,True,4,4,Carrera de San Jerónimo,Centro,...,True,10780.0,True,True,False,True,"Sol, Madrid",Piso en Carrera de San Jerónimo,flat,None
106822977,235000.0,34,bj,studio,38.0,False,0,1,calle de San Andrés,Centro,...,True,6184.0,True,True,False,True,"Malasaña-Universidad, Madrid",Estudio en calle de San Andrés,flat,studio


### Creación nueva variable IMPORTANTE: Terraza
(correspondería en el esquema al punto 13.3)

Se pretende considerar el hecho de que un piso tenga o no terraza como un elemento de estudio en los datos. Sin embargo, este dato no se obtiene en el scrapping de Idealista. Así pues, se busca y sondea en la descripción del inmueble, con la seguridad de que si el piso tiene terraza, y salvo alguna excepción (terraza mínima, residual o trastero), esta estará citada en dicha descripción. 
Se asume cierto error en esta asunción, pero por contra, es seguro que mejorará las predicciones.

In [15]:
# Mostrar todas las filas donde 'terraza' aparece en Descripción
pd.set_option('display.max_colwidth', None)  # No limitar ancho de columna
pd.set_option('display.max_rows', None)      # Mostrar todas las filas
print(len(df2[df2['description'].str.contains('terraza', case=False, na=False)]))
df2[df2['description'].str.contains('terraza', case=False, na=False)][['propertyType', 'description']]

263


,propertyType,description
ID,,
102711209,flat,"Propiedades con terraza en zona prime de Centro Engel & Völkers presenta estas magníficas propiedades en zona prime de Centro. Se encuentran en un edificio clásico de 1920 rehabilitado íntegramente en el año 2003, para actualizarlo dotándolo de todo tipo de comodidades. Las propiedades han sido completamente reformadas con calidades de lujo, cuidando los más mínimos detalles en sus acabados y diseño y aprovechando al máximo su espacio. Cada una de las dos unidades que la forman cuenta con amplio salón comedor y cocina abierta, dos dormitorios y dos baños, uno de ellos en suite y una amplia terraza. Tras pasar el umbral de la de mayor tamaño nos espera la amplia y diáfana zona de salón con techos altos, que cuenta con un comedor parcialmente separado de ella que se abre a la magnífica terraza, y la cocina abierta, separada igualmente por dos prácticas penínsulas. Está totalmente equipada con electrodomésticos de primera calidad. La zona privada nos lleva a los dos dormitorios de los que dispone, uno de ellos con baño en suite y un segundo que da servicio al segundo dormitorio. Ambos dormitorios cuentan con amplios armarios empotrados, que se unen a los que encontramos en el pasillo. La segunda unidad, con puerta independiente, pero conectada por la terraza gracias a un estiloso cerramiento, cuenta en la zona de acceso con una práctica área de trabajo con vistas a la terraza y una cómoda barra de desayunos, igualmente luminosa por su apertura visual a la terraza. Esta zona pública se completa con el espacio ocupado por el salón con cocina abierta totalmente equipada. Tras ella nos encontramos el área privada que incluye igualmente dos dormitorios, uno de ellos con baño en suite, además de un segundo baño. La unidad dispone de numerosos armarios empotrados y un área de lavandería y almacenaje. Unas propiedades tremendamente atractivas por su excelente ubicación, calidades y comodidad, no sólo para inversores sino también para quienes quieran vivir en zona prime del centro de Madrid, y disfrutar de algunos de los principales centros de interés de la capital a tan sólo unos minutos a pié. Las propiedades se encuentran ubicadas en una zona prime del centro de Madrid, al situarse a escasos metros de la estilosa Plaza de Canalejas y de la popular Puerta del Sol, y a escasos minutos andando de algunas de las arterias más emblemáticas de la ciudad, como la Gran Vía o el Paseo del Prado. El barrio cuenta con una amplísima dotación de todos los servicios. En el plano cultural, nos encontramos con el Ateneo de Madrid, el Teatro Español y el Teatro Reina Victoria, entre otros. A nivel comercial, la oferta es inmejorable, ya que incluye desde grandes almacenes a establecimientos centenarios, sin olvidar las sucursales de la mayoría de las marcas de moda nacionales e internacionales. La selección de alojamientos, es igualmente amplia y comprende tanto hoteles gran lujo como alternativas más económicas. La propuesta gastronómica es de lo más variada e internacional también. En este sentido, el nuevo Centro Canalejas, icono del lujo en el corazón de la capital, se encuentra a escasos metros. En materia de comunicaciones, dispone de numerosas líneas de autobuses y varias estaciones de la red de metro y de Cercanías, además de varias paradas de taxi. En definitiva, una propiedad excepcional con calidades de lujo en zona prime del centro de Madrid."
105113334,penthouse,"GILMAR CONSULTING INMOBILIARIO presenta a sus clientes, este impresionante ático se encuentra en una de las zonas más exclusivas de Madrid, en el barrio de Malasaña, en el distrito Centro. Con una ubicación privilegiada, este inmueble goza de una gran cercanía a todo lo que necesitas para disfrutar de una vida cómoda y llena de comodidades. El ático ha sido completamente reformado a estrenar, con un estilo moderno y elegante que combina a la perfección con las características originales del edificio, como las hermosas vigas de madera q

Sondeada la columna description, incorporo la nueva columna con todas las cadenas de texto a buscar.

In [ ]:
# Crear patrones de regex para buscar terrazas
patrones_terraza = [
    r', terraza,', 
    r'con terraza',
    r'magnifica terraza',
    r'amplia terraza',
    r'gran terraza',
    r'grandes terrazas',
    r'terraza privada',
    r'una terraza',
    r'dos terrazas',
    r'terraza de \d+ metros',
    r'terraza de \d+ m2'
]

# Combinar todos los patrones en una sola expresión regular
patron_combinado = '|'.join(patrones_terraza)

# Función para detectar si hay mención de terraza según los patrones
def tiene_terraza(texto):
    if pd.isna(texto):
        return 0
    # Convertir a minúsculas para hacer la búsqueda insensible a mayúsculas
    texto = texto.lower()
    return 1 if re.search(patron_combinado, texto) else 0

# Insertar la nueva columna después de 'price'
# Primero obtenemos la posición de la columna 'price'
posicion_price = df2.columns.get_loc('price')

# Creamos la serie con los valores de terraza
serie_terraza = df2['description'].apply(tiene_terraza)

# Insertamos la columna después de 'price'
# Primero creamos una copia del dataframe para no modificar el original
df2_Terraza = df2.copy()

# Insertamos la columna en la posición deseada
cols = list(df2_Terraza.columns)
cols.insert(posicion_price + 1, 'terraza')
df2_Terraza = df2_Terraza.reindex(columns=cols)
df2_Terraza['terraza'] = serie_terraza

# Verificamos algunas filas para comprobar que funciona
df2_Terraza.head()[['price', 'terraza', 'description']]

_____________________________________

In [ ]:
ini_inspec(df2)

In [ ]:
crear_tabla_resumen(df2)

In [ ]:
categoricas(df2)

### Análisis de las variables

A continuación, una rápida analítica de cada una de las variables.
1. **Variable**: nombre variable/alias
2. **Data type**: cualitativa, cuantitativa, ordinal, continua...¿?
3. **Segmento**: clasificar las variables según su significado. Si son variables demográficas, económicas, identificadores, tiempo...
4. **Expectativas**: un pequeño indicador personal de si resultará útil la variable. ¿Necesito esta variable para la solución? ¿Cómo de importante será esta variable? ¿Esta info la recoge otra variable ya vista?
5. **Conclusiones**: después del análisis anterior, llegar a unas conclusiones sobre la importancia de la variable.

|Variable |Dtype |tipo |faltantes |segmento |expectativas |conclusiones|
|--|--|--|--|--|--|--|
|||||unidades |descripción |
|propertyCode(ID)| int64| entero|código numérico Idealista||ID
|numPhotos|int64 | entero||ud.|posible categorizador|probar| 
|floor |object | float(discreto)| 64|piso|valiosa|importante|
|price | float64| continuo||€|predicción|target|
|propertyType| object |categórico|
|size| float64| continuo|
|exterior| object| booleano |24|bool|valioso|importante|
|rooms| int64| entero|||||
|bathrooms| int64| entero|
|address| object| categórico|
|district| object| categórico|
|neighborhood| object| categórico|
|latitude|float64 | continuo|
|longitude|float64 | continuo|
|description| object| categórico|
|hasVideo| bool| booleano |
|status| object| categórico|5|valoración incremental discreta|valiosa|ordinal encoder|
|hasLift|  object| booleano |6| bool|valiosa|importante|
|priceByArea| float64 | continuo|0|€/m2 |colinealidad| a eliminar|
|hasPlan| bool| booleano |
|has3DTour|bool| booleano |
|has360| bool| booleano |
|topPlus| bool| booleano |
|suggestedTexts_subtitle|object| categórico| | ¿datos adicionales?|¿redundante?||
|suggestedTexts_title |object| categórico| | ¿datos adicionales?|¿redundante?||
|detailedType_typology |   object| categórico| | clasificación|¿redundante?|eliminar|
|detailedType_subTypology | object| categórico|1524| clasificación|¿redundante?|eliminar|

[Ir al inicio de la sección](#comprension-variables)


<a id="reduccion-variables"></a>
## 5. Reducción (tratamiento) de variables preliminar

En este caso que estamos trabajando solo con el distrito centro, puedo eliminara la columna 'district'. En caso de trabajar con varios distritos la mantendría para introducirla en el módelo.

In [ ]:
df2.drop('district', axis=1, inplace=True)

### Tratamiento de la columna 'floor'

In [ ]:
df2['floor'].unique()

In [ ]:
print(f"Número de valores faltantes en 'floor': {df2['floor'].isnull().sum()}")
print("Filas con valores faltantes en 'floor':")
df2[df2['floor'].isnull()].head().T


In [ ]:
# Ajustar pandas para que no trunque el texto
pd.set_option('display.max_colwidth', None)

# Filtrar el DataFrame para obtener las filas donde 'floor' tiene valores NaN
df2[df2['floor'].isnull()][['floor','description']]

In [ ]:
# Crear una lista con los IDs donde 'floor' es NaN
ids_floor_nan = df2[df2['floor'].isnull()].index.tolist()

# Guardar la lista de IDs en una variable para uso futuro
print(f"Lista de IDs con 'floor' como NaN (longitud {len(ids_floor_nan)}):")
print(ids_floor_nan)

Esta función de imputación se basa en busca piso en la descripción, asumiendo que en el distrito centro no existen muchos bloques de más de 5 alturas, y es esos casos excepcionales, es seguro que hubieran indicado explícitamente el piso en su campo.

In [ ]:
def imputar_floor(dataframe):
    """
    Imputa valores en la columna 'floor' basándose en palabras clave encontradas en la columna 'description'
    para las filas donde 'floor' es NaN.

    :param dataframe: DataFrame que debe contener las columnas 'floor' y 'description'.
    :return: DataFrame con los valores imputados en 'floor'.
    """
    # Diccionario que mapea palabras clave a valores de 'floor'
    floor_mapping = {
        1: ['1º', '1ª', 'primer piso', 'piso primero', 'planta primera', 'primera planta'],
        2: ['2º', '2ª', 'segundo piso', 'piso segundo', 'planta segunda', 'segunda planta'],
        3: ['3º', '3ª', 'tercer piso', 'piso tercero', 'tercera planta'],
        4: ['4º', '4ª', 'cuarto piso', 'cuarta planta'],
        5: ['5º', '5ª', 'quinto piso', 'quinta planta'],
        6: ['6º', '6ª','sexto piso', 'sexta planta']
    }

    # Filtrar las filas donde 'floor' es NaN
    filtro = dataframe[dataframe['floor'].isnull()]

    # Iterar sobre el filtro para verificar palabras clave
    for index, row in filtro.iterrows():
        descripcion = str(row['description']).lower()  # Convertir a minúsculas
        for floor, keywords in floor_mapping.items():
            # Verificar si alguna palabra clave está en la descripción
            if any(keyword in descripcion for keyword in keywords):
                dataframe.at[index, 'floor'] = floor  # Asignar el valor correspondiente
                break  # Romper el bucle después de encontrar una coincidencia

    return dataframe

In [ ]:
imputar_floor(df2)

In [ ]:
df2['floor'].isnull().sum()

In [ ]:
df2['floor'].unique()

In [ ]:
# Mostrar el resultado de la lista de las filas imputadas
df2.loc[ids_floor_nan, ['floor','description']]

In [ ]:
# Imputar el resto de NaNs restante a 0
df2['floor'] = df2['floor'].fillna(0)

### Tratamiento de la columna 'exterior'

In [ ]:
df2['exterior'].unique()

In [ ]:
pd.set_option('display.max_colwidth', None)
# Filtrar el DataFrame para obtener las filas donde 'exterior' tiene valores NaN
df2[df2['exterior'].isnull()][['exterior','description']]

In [ ]:
# Crear una lista con los IDs donde 'exterior' es NaN
ids_exterior_nan = df2[df2['exterior'].isnull()].index.tolist()

# Guardar la lista de IDs en una variable para uso futuro
print(f"Lista de IDs con 'exterior' como NaN (longitud {len(ids_exterior_nan)}):")
print(ids_exterior_nan)

In [ ]:
def imputar_exterior(dataframe):
    """
    Esta función recorre un DataFrame y realiza imputaciones en la columna 'exterior' basándose
    en las expresiones encontradas en la columna 'description'.
    
    - Si la descripción contiene expresiones positivas, reemplaza NaN en 'exterior' por True.
    - Si la descripción contiene expresiones negativas, reemplaza NaN en 'exterior' por False.
    - Si no se encuentra ninguna coincidencia, también reemplaza NaN por False.
    
    :param dataframe: DataFrame que debe contener las columnas 'exterior' y 'description'.
    :return: DataFrame con los valores imputados en 'exterior'.
    """
    # Listas de expresiones a buscar
    expresiones_positivas = ['es exterior', 'exterior', 'piso exterior']
    expresiones_negativas = ['interior', 'es interior', 'no es exterior']

    # Filtrar las filas donde 'exterior' es NaN
    filtro = dataframe[dataframe['exterior'].isnull()]

    # Iterar sobre el filtro para verificar y reemplazar en el DataFrame original
    for index, row in filtro.iterrows():
        descripcion = str(row['description']).lower()
        if any(neg in descripcion for neg in expresiones_negativas):
            # Si hay una expresión negativa, reemplazar NaN por False
            dataframe.at[index, 'exterior'] = False
        elif any(pos in descripcion for pos in expresiones_positivas):
            # Si hay una expresión positiva, reemplazar NaN por True
            dataframe.at[index, 'exterior'] = True
        else:
            # Si no hay coincidencias, reemplazar NaN por False
            dataframe.at[index, 'exterior'] = False

    return dataframe


In [ ]:
imputar_exterior (df2)

In [ ]:
# Mostrar la lista de las filas imputadas
df2.loc[ids_exterior_nan, ['exterior']]

In [ ]:
df2['exterior'].unique()

### Tratamiento de la columna 'status'

In [ ]:
df2['status'].unique()

In [ ]:
# Crear una lista con los IDs donde 'status' es NaN
ids_status_nan = df2[df2['status'].isnull()].index.tolist()

# Guardar la lista de IDs en una variable para uso futuro
print(f"Lista de IDs con 'status' como NaN (longitud {len(ids_status_nan)}):")
print(ids_status_nan)

In [ ]:
pd.set_option('display.max_colwidth', None)
# Filtrar el DataFrame para obtener las filas donde 'exterior' tiene valores NaN
df2[df2['status'].isnull()][['status','description']]

Aquí se comprueba que quiza Idealista no permita indicar (o no se desea) Rehabilitación integral o Rehabilitación o Proyecto como un status válido, y aparece en la descripción. Se buscan dichas palabras, y en el caso de aparecer, se crean etiquetas que luego se convertirán en números. Si no se encuentra, se omite y se presupone la peor circunstancia, esto es 'Reformar'

In [ ]:
def imputar_status(dataframe):
    """
    Imputa valores en la columna 'status' basándose en palabras clave encontradas en la columna 'description'
    para las filas donde 'status' es NaN.

    :param dataframe: DataFrame que debe contener las columnas 'status' y 'description'.
    :return: DataFrame con los valores imputados en 'status'.
    """
    # Diccionario que mapea palabras clave a valores de 'status'
    status_mapping = {
        'Nueva': ['obra nueva', 'proyecto'],
        'Rehab': ['rehabilitado', 'rehabilitación']
    }

    # Filtrar las filas donde 'status' es NaN
    filtro = dataframe[dataframe['status'].isnull()]

    # Iterar sobre el filtro para verificar palabras clave
    for index, row in filtro.iterrows():
        descripcion = str(row['description']).lower()  # Convertir a minúsculas
        encontrado = False  # Bandera para saber si se asignó un valor
        for status, keywords in status_mapping.items():
            # Verificar si alguna palabra clave está en la descripción
            if any(keyword in descripcion for keyword in keywords):
                dataframe.at[index, 'status'] = status  # Asignar el valor correspondiente
                encontrado = True
                break  # Romper el bucle después de encontrar una coincidencia
        if not encontrado:
            # Si no se encontró ninguna palabra clave, se asume como la situación mas desfavorable, asignar 'Reformar'
            dataframe.at[index, 'status'] = 'Reformar'

    return dataframe


In [ ]:
imputar_status(df2)

In [ ]:
# Mostrar la lista de las filas imputadas
df2.loc[ids_status_nan, ['status']]

### Tratamiento de la columna 'lift'

In [ ]:
df2['hasLift'].unique()

In [ ]:
# Crear una lista con los IDs donde 'hasLift' es NaN
ids_lift_nan = df2[df2['hasLift'].isnull()].index.tolist()

# Guardar la lista de IDs en una variable para uso futuro
print(f"Lista de IDs con 'hasLift' como NaN (longitud {len(ids_lift_nan)}):")
print(ids_lift_nan)

In [ ]:
# Ajustar pandas para que no trunque el texto
pd.set_option('display.max_colwidth', None)

# Filtrar el DataFrame para obtener las filas donde 'lift' tiene valores NaN
df2[df2['hasLift'].isnull()][['hasLift','floor','description']]

Este tratamiento parte de la base de que en caso de no indicar si tiene ascensor (NaNs) y no decirlo clara y explícitamente en la descripción, debemos entender que no tiene ascensor. 

In [ ]:
# Filtrar filas donde 'hasLift' es NaN
filtro = df2[df2['hasLift'].isnull()]

# Listas de expresiones a buscar
expresiones_positivas = ['tiene ascensor', 'dispone de ascensor', 'con ascensor']
expresiones_negativas = ['no tiene ascensor', 'no dispone de ascensor', 'sin ascensor']

# Iterar sobre el filtro para verificar y reemplazar en el DataFrame original
for index, row in filtro.iterrows():
    descripcion = str(row['description']).lower()
    if any(neg in descripcion for neg in expresiones_negativas):
        # Si hay una expresión negativa, reemplazar NaN por False
        df2.at[index, 'hasLift'] = False
    elif any(pos in descripcion for pos in expresiones_positivas):
        # Si hay una expresión positiva, reemplazar NaN por True
        df2.at[index, 'hasLift'] = True
    else:
        # Si no hay ninguna coincidencia, reemplazar NaN por False
        df2.at[index, 'hasLift'] = False

In [ ]:
df2['hasLift'].unique()

Realizo una función para imputar ascensor:

In [ ]:
def imputar_ascensor(dataframe):
    """
    Esta función recorre un DataFrame y realiza imputaciones en la columna 'hasLift' basándose
    en las expresiones encontradas en la columna 'description'.
    
    - Si la descripción contiene expresiones positivas, reemplaza NaN en 'hasLift' por True.
    - Si la descripción contiene expresiones negativas, reemplaza NaN en 'hasLift' por False.
    - Si no se encuentra ninguna coincidencia, también reemplaza NaN por False.
    
    :param dataframe: DataFrame que debe contener las columnas 'hasLift' y 'description'.
    :return: DataFrame con los valores imputados en 'hasLift'.
    """
    # Listas de expresiones a buscar
    expresiones_positivas = ['tiene ascensor', 'dispone de ascensor', 'con ascensor']
    expresiones_negativas = ['no tiene ascensor', 'no dispone de ascensor', 'sin ascensor']

    # Filtrar las filas donde 'hasLift' es NaN
    filtro = dataframe[dataframe['hasLift'].isnull()]

    # Iterar sobre el filtro para verificar y reemplazar en el DataFrame original
    for index, row in filtro.iterrows():
        descripcion = str(row['description']).lower()
        if any(neg in descripcion for neg in expresiones_negativas):
            # Si hay una expresión negativa, reemplazar NaN por False
            dataframe.at[index, 'hasLift'] = False
        elif any(pos in descripcion for pos in expresiones_positivas):
            # Si hay una expresión positiva, reemplazar NaN por True
            dataframe.at[index, 'hasLift'] = True
        else:
            # Si no hay coincidencias, reemplazar NaN por False
            dataframe.at[index, 'hasLift'] = False

    return dataframe


In [ ]:
imputar_ascensor(df2)

In [ ]:
# Mostrar la lista de las filas imputadas
df2.loc[ids_lift_nan, ['hasLift']]

### Tratamiento de la columna 'detailedType_subTypology'

In [ ]:
df2['detailedType_subTypology'].unique()

In [ ]:
df2[['propertyType', 'detailedType_typology', 'detailedType_subTypology']]

Estudiadas las tres columnas que detallan tipo de propiedad, imputar los nulos de subTypology tendría el mismo resultado que usar propertyType, así que para estos datos donde todos los valores de detailedType son 'flat', decido eliminar en este momento detailedType, ambas columnas.

In [ ]:
df3 = df2.drop(['detailedType_typology', 'detailedType_subTypology'],axis=1)

In [ ]:
df3.head(1)

In [ ]:
crear_tabla_resumen (df3)

###
Eliminados todos los nulos, algo que funcionaría relativamente bien en nuevos datos, pasamos a realizar algunas relaciones y estudio de variables.

### Transformaciones en el tipo de datos

 Primeramente forzar el tipo del dato

In [ ]:
def forzar_data_type(dataframe):
    """
    Fuerza los tipos de datos en las columnas de un DataFrame según un mapeo predefinido.
    
    Tipos:
    - int: numPhotos, rooms, bathrooms
    - float: price, size, latitude, longitude, priceByArea
    - str: [address, description, suggestedTexts_subtitle, suggestedTexts_title
    - pd.StringDtype: floor, propertyType district, neighborhood, status
    - bool: exterior, hasVideo, hasLift, hasPlan, has3DTour, has360, topPlus
    
    :param dataframe: DataFrame que será modificado.
    :return: DataFrame con los tipos de datos forzados.
    """
    # Diccionario con tipo de dato como clave y lista de columnas como valor
    new_types = {
        int: ['numPhotos', 'rooms', 'bathrooms'],
        float: ['price', 'size', 'latitude', 'longitude', 'priceByArea'],
        str: ['address', 'description', 'suggestedTexts_subtitle', 'suggestedTexts_title'],
        pd.StringDtype(): ['floor', 'propertyType','district', 'neighborhood', 'status'],   #es lo mismo que poner "string:....."
        bool: ['exterior', 'hasVideo', 'hasLift', 'hasPlan', 'has3DTour', 'has360', 'topPlus']
    }
    
    # Iterar sobre el diccionario y aplicar el tipo de dato a las columnas especificadas
    for data_type, columns in new_types.items():
        for column in columns:
            if column in dataframe.columns:
                # Limpieza previa para evitar errores de conversión
                if data_type == str:
                    dataframe[column] = dataframe[column].fillna('').astype(str)
                elif data_type == StringDtype():
                    dataframe[column] = dataframe[column].fillna('').astype(pd.StringDtype())  
                elif data_type in [int, float]:
                    dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')
                elif data_type == bool:
                    dataframe[column] = dataframe[column].astype(bool)
    dataframe.info()
    return dataframe


In [ ]:
forzar_data_type (df3)

Tengo 24 variables. Sigo las posibles caminos para reducir las variables. Comprobado que no tengo (5.1.) columnas con missing, (5.2.) variables repetidas, ni columnas con altísima cardinalidad, (5.3.), identificadores o valores únicos, que no aportan nada, paso a la **selección de variables**

### 5.4 Selección de variables

Voy a realizar una matriz de correlación para visualizar las relaciones entre ellas, previamente a la realización de pruebas matemáticas para intentar eliminar alguna de las columnas que puedan ser irrelevantes para mi target.

In [ ]:
# Matriz de correlación SOLO con las columnas numéricas
columnas_numericas = df3.select_dtypes(include=['float64', 'int64']).columns

# Crear matriz de correlación solo con las columnas numéricas
matriz_corr = df3[columnas_numericas].corr()

# Ordenar las columnas en función de su correlación con 'price'
matriz_corr_target = matriz_corr['price'].sort_values(ascending=False)

# Visualizar como un heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Matriz de Correlación (variables numéricas)', fontsize=14)
plt.show()

# Mostrar correlación específica de las variables con el target 'price'
print("Correlación de cada variable con 'price':\n")
print(matriz_corr_target)


In [ ]:
# Función para matriz de correlación
def cramer(x, y):
    """Calcula el coeficiente de Cramer para dos variables categóricas."""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def correlacion(dataframe, booleanas=None, categoricas=None, mostrar_valores=True):
    """
    Calcula y grafica la matriz de correlación de un DataFrame con opciones para incluir booleanos y categóricas.
    
    Parámetros:
    - dataframe: DataFrame con los datos.
    - booleanas: Si 'bool', convierte booleanos a dummies (0/1) para incluirlos en la correlación.
    - categoricas: Si 'Cramer', calcula la correlación entre variables categóricas usando Cramer’s V.
    - mostrar_valores: por defecto True, muestra los valores numéricos en el heatmap.
    
    Devuelve:
    - La matriz de correlación calculada.
    """
    
    df = dataframe.copy()
    
    # Convertir booleanos a numéricos si 'booleanas' es 'bool'
    if booleanas == 'bool':
        bool_cols = df.select_dtypes(include=['bool']).columns
        df[bool_cols] = df[bool_cols].astype(int)  # Convertir a 0/1
    
    # Seleccionar solo columnas numéricas
    columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
    matriz_corr = df[columnas_numericas].corr()
    
    # Si se pide incluir categóricas con Cramer’s V
    if categoricas == 'Cramer':
        cat_cols = df.select_dtypes(include=['object', 'category']).columns
        if len(cat_cols) > 0:
            cramer_corr = pd.DataFrame(index=cat_cols, columns=cat_cols)
        
        for col1 in cat_cols:
            for col2 in cat_cols:
                if col1 == col2:
                    cramer_corr.loc[col1, col2] = 1  # Autocorrelación
                else:
                    cramer_corr.loc[col1, col2] = cramer(df[col1], df[col2])
        
        cramer_corr = cramer_corr.astype(float)
        matriz_corr = matriz_corr.combine_first(cramer_corr)  # Unir ambas matrices

    # Ordenar las columnas en función de su correlación con 'price'
    if 'price' in matriz_corr.columns:
        matriz_corr_target = matriz_corr['price'].sort_values(ascending=False)
        print("\nCorrelación de cada variable con 'price':\n")
        print(matriz_corr_target)
    
    # Graficar heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(matriz_corr, annot=mostrar_valores, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Matriz de Correlación', fontsize=14)
    plt.show()
    
    return matriz_corr


In [ ]:
correlacion (df3)

In [ ]:
#Corrrelación limitada a price, priceByArea y size
# Seleccionar las columnas de interés
# cols = ['price', 'priceByArea', 'size']

# # Calcular la matriz de correlación
# corr_matrix = df3[cols].corr()

# # Visualizar la matriz de correlación con un heatmap
# plt.figure(figsize=(6, 4))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title("Matriz de Correlación entre Price, PriceByArea y Size")
# plt.show()


Si bien fruto de la anterior matriz de correlación, procedería eliminar size como la que menos información aporta y alta correlación con price, parece bastante trampa, pues es un dato que se obtiene teniendo el precio de partida. Como esto no sucederá en datos futuros ni en test, y no debe suceder en validation, voy a eliminarla en este momento

In [ ]:
df3.drop('priceByArea', axis=1, inplace=True)

#### Conversiones numéricas básicas

In [ ]:
df3_num = df3.copy()

##### Conversión numérica mediante mapeo de 'floor'


In [ ]:
# Celda para encontrar detalle del significado de 'st' en 'floor' en la columna descripción
# Ajustar pandas para que no trunque el texto
pd.set_option('display.max_colwidth', None)

# Filtrar las filas donde 'floor' es igual a 'st'
df3[df3['floor'] == 'st'][['floor', 'description']]


In [ ]:
def convertir_floor_flotante(dataframe):
    """
    Convierte la columna 'floor' a valores numéricos según un mapeo predefinido.
    
    - Si el valor ya es numérico, se mantiene.
    - Si es un texto reconocido ('bj', 'ss', etc.), se mapea a un número.
    - Si es un número en texto ('3', '-1', etc.), se convierte a número.
    - Si es un valor desconocido o vacío, se asigna 0.

    :param dataframe: DataFrame con la columna 'floor'.
    :return: DataFrame con 'floor' en formato numérico.
    """
    # Diccionario de mapeo
    floor_mapping = {
        'entreplanta': 0.5, 'ent': 0.5, 'en': 0.5,
        'baja': 0, 'bajo': 0, 'bj': 0, 'street': 0, 'st': 0,
        'semisótano': -0.5, 'semisotano': -0.5, 'ss': -0.5,
        'sótano': -1, 'sotano': -1, 'sot': -1
    }

    def map_floor(value):
        # Si ya es numérico, se mantiene
        if isinstance(value, (int, float)):
            return value

        # Si el valor es NaN o None, asignar 0
        if pd.isna(value):
            return 0

        # Convertir a string y limpiar espacios
        value = str(value).strip().lower()

        # Si está en el diccionario, aplicar el mapeo
        if value in floor_mapping:
            return floor_mapping[value]

        # Intentar convertir números en string ('3', '-1', etc.)
        try:
            return int(value) if value.isdigit() or value.lstrip('-').isdigit() else float(value)
        except ValueError:
            return 0  # Valores desconocidos se asignan a 0

    # Aplicar la función y forzar a numérico
    dataframe['floor'] = dataframe['floor'].apply(map_floor)

    return dataframe


In [ ]:
df3_num = convertir_floor_flotante(df3_num)
df3_num.head(5)

In [ ]:
df3_num['floor'].unique()

##### Conversión numérica mediante mapeo de 'status'

In [ ]:
df3_num['status'].unique()

In [ ]:
def mapear_status(dataframe):
    """
    Convierte la columna 'status' a valores numéricos según un mapeo predefinido.
    
    - 'Nueva' y 'newdevelopment' → 2
    - 'good' → 1
    - 'renew' → 0
    - Cualquier otro valor (incluyendo NaN) → 0

    :param dataframe: DataFrame con la columna 'status'.
    :return: DataFrame con 'status' en formato numérico.
    """
    # Diccionario de mapeo
    status_mapping = {
        'nueva': 2, 'newdevelopment': 2,
        'good': 1,
        'renew': 0
    }

    def map_status(value):
        # Si ya es numérico, se mantiene
        if isinstance(value, (int, float)):
            return value

        # Si el valor es NaN o None, asignar 0
        if pd.isna(value):
            return 0

        # Convertir a string y limpiar espacios
        value = str(value).strip().lower()

        # Si está en el diccionario, aplicar el mapeo
        return status_mapping.get(value, 0)

    # Aplicar la función y forzar a numérico
    dataframe['status'] = dataframe['status'].apply(map_status)

    return dataframe


In [ ]:
df3_num= mapear_status (df3_num)

In [ ]:
df3_num['status'].unique()

##### Conversión automática resto de columnas binarias y (algunas) categóricas 

In [ ]:
df3_num['propertyType'].unique()

In [ ]:
def convertir_a_numerico (dataframe):
    """
    Convierte los valores del DataFrame:
    - Convierte columnas booleanas a 0 y 1.
    - Renombra 'district' → 'distrito' y 'neighborhood' → 'barrio'.
    - Usa one-hot encoding para 'district' y 'neighborhood', si existen en el DataFrame.
    - Renombra 'propertyType' → 'tipo' y usa one-hot encoding para 'tipo'.

    :param dataframe: DataFrame de entrada.
    :return: DataFrame transformado.
    """
    df_numerico = dataframe.copy()

    # Convertir booleanos a 0 y 1
    bool_cols = ['exterior', 'hasVideo', 'hasLift', 'hasPlan', 'has3DTour', 'has360', 'topPlus']
    for col in bool_cols:
        if col in df_numerico.columns:
            df_numerico[col] = df_numerico[col].astype(int)
            
    # Renombrar columnas antes de cualquier otra transformación
    df_numerico = df_numerico.rename(columns={'district': 'distrito', 'neighborhood': 'barrio' , 'propertyType': 'tipo'})

    # Aplicar one-hot encoding a 'district' y 'neighborhood' si existen
    for col in ['districto', 'barrio']:
        if col in df_numerico.columns:
            dummies = pd.get_dummies(df_numerico[col], prefix=col, dtype=int)
            df_numerico = pd.concat([df_numerico.drop(columns=[col]), dummies], axis=1)

    # Aplicar one-hot encoding a 'tipo' para las categorías especificadas
    if 'tipo' in df_numerico.columns:
        tipos = ['flat', 'studio', 'penthouse', 'duplex']
        # Verificar que las categorías estén en la columna 'tipo'
        for tipo in tipos:
            if tipo in df_numerico['tipo'].values:
                # Crear columnas one-hot para cada tipo
                df_numerico[f'tipo_{tipo}'] = (df_numerico['tipo'] == tipo).astype(int)
        # Borrar la columna original 'tipo' después del one-hot encoding
        # df_numerico = df_numerico.drop(columns=['tipo']) #no la borro, para el estudio de outliers. La borraré después
        
    return df_numerico


In [ ]:
# Aplicar la conversión
df3_num = convertir_a_numerico(df3_num)
df3_num.head()

In [ ]:
numericas (df3_num)

In [ ]:
crear_tabla_resumen (df3_num)

In [ ]:
# Con los datos, función para convertir en csv y guardarlo.
today =  date.today ()
file_path = f'../data/processed/ide_viv_numerico0_{today}.csv' 

def df_to_csv(df):
    if os.path.exists(file_path):
        print(f"⚠️ El archivo '{file_path}' ya existe. No se sobrescribirá.")
    else:
        df.to_csv(file_path)   #lo guarda en un csv con indice en propertyCode
        print(f"✅ Archivo guardado correctamente como '{file_path}'.")

In [ ]:
#Guarda los datos pre separación Train-test en un csv con nombre establecido.
df_to_csv(df3_num)

In [ ]:
correlacion (df3_num)

###
[Ir al principio de la sección 5](#reduccion-variables)


<a id="analisis-univariante"></a>
## 6. Análisis univariante

In [ ]:
crear_tabla_resumen (df3)

In [ ]:
# Función para gráficos univariante 

def graficos_uni(dataframe):
    """
    Genera gráficos de distribución:
    - Barras para variables categóricas, discretas y booleanas.
    - Histogramas para variables continuas.

    Parámetro:
    - dataframe: DataFrame con los datos.
    """
    df = dataframe.copy()

    # Excluir columnas no deseadas
    excluidas = {'address', 'description', 'suggestedTexts_title'}
    df = df.drop(columns=[col for col in excluidas if col in df.columns])

    # Clasificar variables
    categ_discretas = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categ_discretas += [col for col in df.select_dtypes(include=['int64']).columns if df[col].nunique() <= 15]
    bool_vars = df.select_dtypes(include=['bool']).columns.tolist()
    continuas = [col for col in df.select_dtypes(include=['float64', 'int64']).columns if col not in categ_discretas]

    num_vars = len(categ_discretas) + len(bool_vars) + len(continuas)
    filas = -(-num_vars // 2)  # Redondeo hacia arriba

    fig, axes = plt.subplots(filas, 2, figsize=(15, filas * 4))
    axes = axes.flatten()

    i = 0
    for col in categ_discretas:
        sns.countplot(data=df, x=col, hue=col, ax=axes[i], legend=False,palette= 'light:gray')
        axes[i].set_title(f"Distribución de {col}")
        axes[i].tick_params(axis='x', rotation=45)
        i += 1

    for col in bool_vars:
        sns.countplot(data=df, x=col, hue=col, ax=axes[i], palette={False: 'lightcoral', True: 'lightgreen'}, legend=False)
        axes[i].set_title(f"Distribución de {col}")
        i += 1

    for col in continuas:
        sns.histplot(df[col], kde=True, ax=axes[i], bins=30, color="gray", edgecolor="black")
        sns.kdeplot(df[col], ax=axes[i], color="red", linewidth=2)
        axes[i].set_title(f"Distribución de {col}")
        i += 1

    for j in range(i, len(axes)):  # Ocultar ejes vacíos
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()


In [ ]:
graficos_uni (df3)

[ir al inicio de la sección 6](#analisis-univariante)

<a id="analisis-bivariante"></a>
## 7. Análisis bivariante

In [ ]:
df3_num.shape

In [ ]:
# Asegúrate de tener solo las variables numéricas, excluidas categóricas
df3_pair = df3.select_dtypes(include=['float64', 'int64'])

# Usamos pairplot con el target 'price' para diferenciar por colores
plt.figure(figsize=(12,8))
sns.pairplot(df3_pair, diag_kind='hist')
#sns.pairplot(df3_pair, kind='scatter')
plt.show()

In [ ]:
df3_pair.shape

In [ ]:
plt.figure(figsize=(16, 6))  # Gráfico más grande para evitar achatamiento

# Boxplot con mejor ajuste de tamaño
sns.boxplot(data=df3, x='propertyType', y='price', hue='propertyType', palette='coolwarm', width=0.6)

# Ajustes estéticos
plt.title('Distribución de precios según tipo de vivienda', fontsize=16)
plt.xlabel('Tipo de vivienda', fontsize=14)
plt.ylabel('Precio (€)', fontsize=14)
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario

# Evitar notación científica en el eje Y
plt.ticklabel_format(style='plain', axis='y')

# Agregar una cuadrícula horizontal para mejor referencia
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Guardar imagen
plt.savefig('graf/boxplot_vivienda_precio.png', dpi=150, bbox_inches='tight')

# Mostrar gráfico
plt.show()


In [ ]:
df3[df3['price'] > 6_000_000]



In [ ]:
df3 = df3[df3.index != 106035767]
df3_num = df3_num[df3_num.index != 106035767]



In [ ]:
plt.figure(figsize=(16, 6))  # Gráfico más grande para evitar achatamiento

# Boxplot con mejor ajuste de tamaño
sns.boxplot(data=df3, x='propertyType', y='price', hue='propertyType', palette='coolwarm', width=0.6)

# Ajustes estéticos
plt.title('Distribución de precios según tipo de vivienda', fontsize=16)
plt.xlabel('Tipo de vivienda', fontsize=14)
plt.ylabel('Precio (€)', fontsize=14)
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario

# Evitar notación científica en el eje Y
plt.ticklabel_format(style='plain', axis='y')

# Agregar una cuadrícula horizontal para mejor referencia
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Guardar imagen
plt.savefig('graf/boxplot_vivienda_precio.png', dpi=150, bbox_inches='tight')

# Mostrar gráfico
plt.show()


[ir al inicio de la sección 7](#analisis-bivariante)

<a id="eliminacion-variables"></a>  
## 8. Eliminación de variables

In [ ]:
crear_tabla_resumen(df3_num)

Vamos a limpiar nuestros datos, después de analizados. Eliminaremos aquellas columnas de texto largo, descripción, dirección y título. También aquellas con alta cardinalidad, latitud y longitud. 

In [ ]:
df3[['suggestedTexts_subtitle', 'neighborhood']]


Comprobado que ambas columnas contienen los mismos datos, para mis efectos, eliminaré también la columna suggestedTexts_subtitle

In [ ]:
def eliminar_variables(dataframe):
    """
    Elimina las columnas de texto largo (descripcion, direccion, etc.) y columnas con alta cardinalidad 
    como latitud y longitud.
    
    :param dataframe: DataFrame de entrada.
    :return: DataFrame sin las columnas no deseadas.
    """
    df_limpio = dataframe.copy()
    
    # Eliminar columnas de texto largo
    columnas_texto_largo = ['description', 'address', 'suggestedTexts_title' ,'suggestedTexts_subtitle']
    df_limpio = df_limpio.drop(columns=[col for col in columnas_texto_largo if col in df_limpio.columns])
    
    # Eliminar columnas con alta cardinalidad (latitud y longitud, o más de 100 valores únicos)
    columnas_alta_cardinalidad = ['latitude', 'longitude']
    
    # También podemos eliminar aquellas columnas con más de 100 valores únicos (como un criterio genérico de alta cardinalidad)
    for col in df_limpio.select_dtypes(include=['object']).columns:
        if df_limpio[col].nunique() > 100:  # Umbral de cardinalidad alta
            columnas_alta_cardinalidad.append(col)

    # Eliminar las columnas con alta cardinalidad
    df_limpio = df_limpio.drop(columns=[col for col in columnas_alta_cardinalidad if col in df_limpio.columns])

    return df_limpio


In [ ]:
df3_num

In [ ]:
df4 = eliminar_variables(df3_num)

In [ ]:
df4.head(2)

In [ ]:
crear_tabla_resumen (df4)

## 9. Duplicados

In [ ]:
df4.duplicated().sum()

Podrían haber duplicados que, bajo diferentes índices, sean el mismo anuncio, bien porque haya varios anunciantes para el mismo piso, o bien porque el mismo anunciante duplique su anuncio publicado.

In [ ]:
# Mostrar todas las filas duplicadas agrupadas
duplicados = df4[df4.duplicated(keep=False)]  # keep=False para marcar todas las duplicadas, no solo las subsecuentes

# Ordenar las filas duplicadas para visualizarlas juntas
duplicados = duplicados.sort_values(by=list(df4.columns))  # Ordenar por todas las columnas

duplicados



In [ ]:
df4 = df4.drop_duplicates(keep='last')


In [ ]:
df4.duplicated().sum()

## 10. Valores faltantes

In [ ]:
print(df4.isnull().sum())

## 11. Anomalías y errores

In [ ]:
crear_tabla_resumen(df4)

In [ ]:
numericas (df4)

In [ ]:
categoricas(df4)

No se observan

<a id="outliers"></a> 
## 12. Outliers

Ya elimine uno clarísimo en el paso de análisis bivariante, ahora voy a ver todos los que están menos claros. 

In [ ]:
# Calcular el percentil 95 para cada tipo de propiedad
percentile_95 = df4.groupby('tipo')['price'].quantile(0.95)

# Mostrar los valores del percentil 95 por tipo de propiedad
print(percentile_95)


Planteo un sistema de eliminar datos fuera de un porcentaje de datos de todos los totales: esto tiene el inconveniente de que no considera la distribución específica de mis datos, y que en distribuciones sesgadas, eliminará datos válidos. Además, trata todos los conjuntos de datos por igual. Eso sí, es simple, directo y predecible.

In [ ]:
# Definir el percentil como variable (por ejemplo, 95% = 0.95)
percentil = 0.95

# Calcular el percentil para cada tipo de propiedad
percentil_valores = df4.groupby('tipo')['price'].quantile(percentil)

# Mostrar los percentiles calculados
print(f"Percentiles al {percentil*100}%:")
print(percentil_valores)

# Calcular el total de datos por tipo de propiedad
total_datos_por_tipo = df4.groupby('tipo').size()

# Calcular el número de datos que están por debajo del percentil para cada tipo de propiedad
datos_percentil = df4[df4['price'] <= df4['tipo'].map(percentil_valores)].groupby('tipo').size()

# Calcular los datos eliminados (total - datos dentro del percentil)
datos_eliminados = total_datos_por_tipo - datos_percentil

# Imprimir la información solicitada
for prop_type in df4['tipo'].unique():
    print(f"\nTipo de vivienda: {prop_type}")
    print(f"Precio percentil {percentil*100}%: {percentil_valores[prop_type]:,.2f}€")
    print(f"Número total de datos iniciales: {total_datos_por_tipo[prop_type]}")
    print(f"Número total de datos en el percentil {percentil*100}%: {datos_percentil[prop_type]}")
    print(f"Número de datos eliminados: {datos_eliminados[prop_type]}")

# Crear el gráfico de boxplot
plt.figure(figsize=(16, 8))  # Gráfico más grande para evitar achatamiento

# Boxplot
sns.boxplot(data=df4, x='tipo', y='price', hue= 'tipo', palette='coolwarm', width=0.6)

# Agregar las líneas horizontales de los percentiles para cada tipo de propiedad
for i, prop_type in enumerate(df4['tipo'].unique()):
    color = plt.cm.coolwarm(i / len(df4['tipo'].unique()))  # Tomar un color de la paleta
    plt.axhline(percentil_valores[prop_type], color=color, linestyle='--', 
                label=f'{prop_type} {percentil*100}%')
    
# Ajustes estéticos
plt.title(f'Distribución de precios según tipo de vivienda (Percentil {percentil*100}%)', fontsize=16)
plt.xlabel('Tipo de vivienda', fontsize=14)
plt.ylabel('Precio (€)', fontsize=14)
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario

# Evitar notación científica en el eje Y
plt.ticklabel_format(style='plain', axis='y')

# Agregar una cuadrícula horizontal para mejor referencia
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Leyenda
plt.legend()

# Guardar imagen
plt.savefig(f'graf/boxplot_vivienda_precio_percentil{percentil*100}.png', dpi=150, bbox_inches='tight')

# Mostrar gráfico
plt.show()


Filtrar por los extremos del boxplot es más adaptado, y es más riguroso. Una vez visto el gráfico anterior, prefiero este sistema, preferible en contextos de análisis inmobiliarios, para evitar distorsionar el modelo con propiedades de lujo o de valor bajo por condiciones singulares (usufructos, expropiaciones, subastas, etc)

In [ ]:
# Selección de outliers por criterio de bigotes.
# Definir el factor de IQR como variable
factor_iqr = 2  # Puedes cambiar este valor según lo necesites

# Calcular los límites de los bigotes del boxplot para cada tipo de propiedad
iqr_limits = {}

for prop_type in df4['tipo'].unique():
    q1 = np.percentile(df4[df4['tipo'] == prop_type]['price'], 25)
    q3 = np.percentile(df4[df4['tipo'] == prop_type]['price'], 75)
    iqr = q3 - q1
    lower_bound = q1 - factor_iqr * iqr
    upper_bound = q3 + factor_iqr * iqr
    iqr_limits[prop_type] = (lower_bound, upper_bound)

# Imprimir el resumen de datos
for prop_type in df4['tipo'].unique():
    lower_bound, upper_bound = iqr_limits[prop_type]
    total_datos_inicial = len(df4[df4['tipo'] == prop_type])
    total_datos_filtrados = len(df4[(df4['tipo'] == prop_type) & 
                                    (df4['price'] >= lower_bound) & 
                                    (df4['price'] <= upper_bound)])
    datos_eliminados = total_datos_inicial - total_datos_filtrados

    print(f"Tipo: {prop_type}, "
          f"Límite inferior: {lower_bound}, "
          f"Límite superior: {upper_bound}, "
          f"Número total de datos inicial: {total_datos_inicial}, "
          f"Número total de datos dentro de los bigotes: {total_datos_filtrados}, "
          f"Número de datos eliminados: {datos_eliminados}")

# Crear el gráfico de boxplot
plt.figure(figsize=(16, 8))  # Gráfico más grande para evitar achatamiento

# Dibujar el boxplot
sns.boxplot(data=df4, x='tipo', y='price', hue='tipo', palette='coolwarm', width=0.6)

# Agregar las líneas horizontales de los bigotes para cada tipo de propiedad
for i, prop_type in enumerate(df4['tipo'].unique()):
    lower_bound, upper_bound = iqr_limits[prop_type]
    color = plt.cm.coolwarm(i / len(df4['tipo'].unique()))  # Tomar un color de la paleta
    plt.axhline(upper_bound, color=color, linestyle='--', label=f'{prop_type} límite superior')
    plt.axhline(lower_bound, color=color, linestyle=':', label=f'{prop_type} límite inferior')

# Ajustes estéticos
plt.title(f'Distribución de precios según tipo de vivienda (Bigotes {factor_iqr}×IQR)', fontsize=16)
plt.xlabel('Tipo de vivienda', fontsize=14)
plt.ylabel('Precio (€)', fontsize=14)
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario

# Evitar notación científica en el eje Y
plt.ticklabel_format(style='plain', axis='y')

# Agregar una cuadrícula horizontal para mejor referencia
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Leyenda
plt.legend()

# Guardar imagen
plt.savefig(f'graf/boxplot_vivienda_precio_IQR{factor_iqr}.png', dpi=150, bbox_inches='tight')

# Mostrar gráfico
plt.show()



Voy a automatizar la eliminación de outliers mediante una función, donde puedo elegir el método y el valor de corte.

In [ ]:
# Función fitrado outliers 
def eliminar_outliers(df, metodo=None, bigotes=1.5, percentil=0.95):
    """
    Filtra outliers de un DataFrame según el método seleccionado.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame de entrada (se copia para no modificar el original).
    - metodo (str): 'bigotes' para usar el método IQR o 'percentil' para usar percentiles.
    - bigotes (float, opcional): Factor multiplicador de IQR para definir los bigotes (default = 1.5).
    - percentil (float, opcional): Percentil para el umbral de eliminación (default = 0.95).
    
    Retorna:
    - pd.DataFrame: Nuevo DataFrame sin los outliers según el método elegido.
    """
    # Copia del DataFrame original
    df_filtrado = df.copy()

    # Verificar si la columna 'tipo' existe
    if 'tipo' not in df.columns or 'price' not in df.columns:
        print("Error: El DataFrame debe contener las columnas 'tipo' y 'price'.")
        return None

    # Si no se ha pasado ningún método, pedirlo
    if metodo not in ['bigotes', 'percentil']:
        print("Debe especificar un método de filtrado: 'bigotes' o 'percentil'.")
        return None

    if metodo == 'bigotes':
        print(f"Filtrando con método 'bigotes' (IQR * {bigotes})...")

        # Calcular los límites de los bigotes para cada tipo de propiedad
        for prop_type in df_filtrado['tipo'].unique():
            subset = df_filtrado[df_filtrado['tipo'] == prop_type]['price']
            q1, q3 = np.percentile(subset, [25, 75])
            iqr = q3 - q1
            lower_bound = q1 - bigotes * iqr
            upper_bound = q3 + bigotes * iqr

            # Filtrar el DataFrame
            df_filtrado = df_filtrado[
                ~((df_filtrado['tipo'] == prop_type) & 
                  ((df_filtrado['price'] < lower_bound) | (df_filtrado['price'] > upper_bound)))
            ]

    elif metodo == 'percentil':
        print(f"Filtrando con método 'percentil' ({percentil*100}%)...")

        # Calcular el percentil seleccionado para cada tipo de propiedad
        percentiles = df_filtrado.groupby('tipo')['price'].quantile(percentil)

        # Filtrar el DataFrame
        df_filtrado = df_filtrado[
            df_filtrado.apply(lambda row: row['price'] <= percentiles[row['tipo']], axis=1)
        ]

    # Mostrar resumen de filtrado
    for prop_type in df['tipo'].unique():
        total_datos_inicial = len(df[df['tipo'] == prop_type])
        total_datos_filtrados = len(df_filtrado[df_filtrado['tipo'] == prop_type])
        datos_eliminados = total_datos_inicial - total_datos_filtrados

        print(f"Tipo: {prop_type}, "
              f"Número total de datos inicial: {total_datos_inicial}, "
              f"Número total de datos filtrados: {total_datos_filtrados}, "
              f"Número de datos eliminados: {datos_eliminados}")

    return df_filtrado


In [ ]:
df4.head()

In [ ]:
print(df4.shape)
df4.columns.tolist

In [ ]:
df5 = eliminar_outliers(df4, metodo='bigotes', bigotes = 2)

In [ ]:
df5.shape

[Ir al inicio de la sección 12 ](#outliers)

<a id="feat-engi"></a>
<a href="#inicio-notebook"><p style="text-align:right;" href="#inicio-notebook">Volver al índice</p></a> 
## 13. Feature Engineering

Aplicación de la transformación logarítmica (probada en el notebook anterior) a la target

In [ ]:
df4_log = df4.copy()  # Crear una copia del DataFrame original
df4_log['price'] = np.log(df4_log['price'])  # Aplicar la transformación logarítmica solo en la copia
df4_log.rename(columns={'price': 'priceLog'}, inplace=True)


In [ ]:

df5_log = df5.copy()  # Crear una copia del DataFrame original
df5_log['price'] = np.log(df5_log['price'])  # Aplicar la transformación logarítmica solo en la copia
df5_log.rename(columns={'price': 'priceLog'}, inplace=True)

In [ ]:
df5_log.head(2)

Eliminamos la columna 'tipo' que mantuvimos para clasificar outliers por grupos

In [ ]:
#eliminar la columna tipo en ambos dataframe df4, df5 a guardar para ML
if 'tipo' in df4.columns:
    df4_log.drop('tipo', axis=1, inplace=True, errors = 'ignore')
else:
    print("⚠️ Advertencia: La columna 'tipo' YA no existe en df4.")

if 'tipo' in df5.columns:
    df5_log.drop('tipo', axis=1, inplace=True, errors = 'ignore')
else:
    print("⚠️ Advertencia: La columna 'tipo' YA no existe en df5.")

In [ ]:
# Datos nombrado archivo.
today =  date.today ()
file_path = f'../data/processed/ide_viv_numerico1_{today}.csv' 

#Guardo df4 como dataframe solo numérico para ML sin reducción de outliers
df_to_csv (df4)

In [ ]:
# Datos nombrado archivo.
today =  date.today ()
file_path = f'../data/processed/ide_viv_numerico2_{today}.csv' 

#Guardo df4 como dataframe solo numérico para ML eliminados outliers
df_to_csv (df5)

## 14. Reducción de dimensionalidad (asistidos)
(basada en procesos automáticos)

Por último probamos antes de pasar a la división de train y validation, feature reduction.

In [ ]:
# Preparamos los datos para la selección de características
X = df5_log.drop(['priceLog'], axis=1)  # Eliminamos la variable objetivo y variables no útiles
y = df5_log['priceLog']

In [ ]:
# Método 1: Correlación con la variable objetivo
correlaciones = X.corrwith(y).abs().sort_values(ascending=False)
print("\nCorrelación de las características con precio:")
print(correlaciones.head(10))

In [ ]:
# Método 2: SelectKBest con f_regression
selector = SelectKBest(score_func=f_regression, k=10)
X_selector = selector.fit_transform(X, y)
mask = selector.get_support()
caracteristicas_seleccionadas = X.columns[mask]

print("\nCaracterísticas seleccionadas con SelectKBest:")
print(caracteristicas_seleccionadas.tolist)

In [ ]:
# Método 3: Importancia de características con Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)
importancias = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

print("\nImportancia de características con Random Forest:")
print(importancias.head(10))

plt.figure(figsize=(12, 6))
importancias.head(10).plot(kind='barh')
plt.title('Top 10 Características más Importantes (Random Forest)')
plt.xlabel('Importancia')
plt.tight_layout()

In [ ]:
# Método 4: RFE con modelo de Random Forest
model = RandomForestRegressor(random_state=42)

# Selección de características con RFE
selector = RFE(model, n_features_to_select=10)
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]

# Obtener la importancia de las características seleccionadas
importances = pd.Series(model.feature_importances_, index=X.columns)

# Ordenar de mayor a menor
importances = importances.sort_values(ascending=False)

# Gráfico de barras con las 10 más importantes
plt.figure(figsize=(12, 6))
importances.head(10).plot(kind='barh', color='royalblue', edgecolor='black')
plt.title('Top 10 Características más Importantes (Random Forest)')
plt.xlabel('Importancia')
plt.gca().invert_yaxis()  # Invertir el eje Y para mostrar la más importante arriba
plt.tight_layout()
plt.show()

# Mostrar las características seleccionadas
print("Características seleccionadas por RFE:")
print(selected_features.tolist())


[Ir al inicio del Notebook](#inicio-notebook)

## 15. División en train y test
(resevar una porción de los datos obtenidos para probar nuestros modelos)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'],
                                                    test_size=0.2,
                                                    random_state=42)
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

___________________________________________
## XX. Transformaciones a aplicar a test (nuevos datos que vengan)

In [ ]:
#agrupación del código a aplicar a test, que apliqué en train

# Detener la ejecución de esta celda

# Todo lo que esté aquí no se ejecutará
print("Esta celda está reservada para aplicar a los datos de test.")
return

# # Definir la columna target
target = 'price'  # Reemplaza con el nombre real de la columna target

# Verificar si la columna target está en el DataFrame y eliminarla si existe
if target in datos_test.columns:
    datos_test = datos_test.drop(columns=[target])

# Continuar con el resto del código sin interrupciones
# df.drop_duplicates(keep='first', inplace=True)    #no es necesario eliminar duplicados en el test



df = datos_test.reset_index(drop=True).set_index("propertyCode")
df.index.name = 'ID'

col_eliminar = ['thumbnail','externalReference', 'priceInfo', 'operation', 'province', 'municipality',
       'country', 'showAddress', 'url', 'newDevelopment', 'change', 'highlight', 'savedAd', 
       'notes','hasStaging', 'topNewDevelopment', 'parkingSpace' ,'newDevelopmentFinished', 'priceByArea']
df.drop(col_eliminar, axis=1, inplace=True)

df = expand_dict_columns(df)

imputar_ascensor(df)






df4.drop('tipo', axis=1, inplace=True, errors = 'ignore')
df5.drop('tipo', axis=1, inplace=True, errors = 'ignore')